In [1]:
# Dependencies and Setup
import os
import pandas as pd
import numpy as np

# File path
csvpath = os.path.join("Resources", "purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
purchaseData_df = pd.read_csv(csvpath)

# Take a look at the first few rows of the data
purchaseData_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# Check for missing values in dataset
purchaseData_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [3]:
# Determine total number of players
totalPlayers = len(purchaseData_df["SN"].unique())

# Display in dataframe
totalPlayers_df = pd.DataFrame({"Total Players": [totalPlayers]})
totalPlayers_df

,Total Players
0,576


In [4]:
# Purchasing Analysis (Total)

# Determine number of unique items purchased
itemCount = len(purchaseData_df["Item ID"].unique())

# Sum prices of all purchases for total revenue
totalRevenue = purchaseData_df["Price"].sum()

# Count number of all purchases
numPurchases = purchaseData_df["Price"].count()

# Average price of all items purchased
meanItemPrice = purchaseData_df["Price"].mean()

# Display in dataframe
purchAnalysisTotal_df = pd.DataFrame({
    "Number of Unique Items": [itemCount], 
    "Average Price": [meanItemPrice], 
    "Number of Purhcases": [numPurchases], 
    "Total Revenue": [totalRevenue]
})

# Check data types in dataframe with purchAnalysisTotal_df.dtypes

# Format dataframe with $
# Note: using pd.options.display.float_format = '${:,.2f}'.format diplays all dataframes with formatting
final_purchTotal_df = purchAnalysisTotal_df.copy()
final_purchTotal_df["Average Price"] = purchAnalysisTotal_df["Average Price"].map("${:,.2f}".format)
final_purchTotal_df["Total Revenue"] = purchAnalysisTotal_df["Total Revenue"].map("${:,.2f}".format)

# Display results in dataframe
final_purchTotal_df

,Number of Unique Items,Average Price,Number of Purhcases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [16]:
# Find highest and lowest price of individual items, and median of total purchases
itemMaxPrice = purchaseData_df["Price"].max()
itemMinPrice = purchaseData_df["Price"].min()
itemMedianPrice = purchaseData_df["Price"].median()
print(itemMaxPrice)
print(itemMinPrice)
print(itemMedianPrice)

# Find name of most and least expensive item
nameItemMax = purchaseData_df.loc[purchaseData_df["Price"] == itemMaxPrice, ["Item Name"]]
print(nameItemMax)

nameItemMin = purchaseData_df.loc[purchaseData_df["Price"] == itemMinPrice, ["Item Name"]]
print(nameItemMin)

4.99
1.0
3.15
          Item Name
189  Stormfury Mace
554  Stormfury Mace
                      Item Name
63   Whistling Mithril Warblade
418  Whistling Mithril Warblade


In [6]:
# Gender Demographics analysis

# New dataframe with names and gender
genderDemo_df = purchaseData_df[["SN", "Gender", "Age"]]

# Remove duplicate names
noDuplicateGender_df = genderDemo_df.drop_duplicates(subset=["SN"])

# Count values for total count per gender and display in dataframe
genderCounts_df = pd.DataFrame(noDuplicateGender_df["Gender"].value_counts())

# Rename column Total Count, calculate % of player by gender, add new column, format with %
genderDemo_df = genderCounts_df.rename(columns={"Gender": "Total Count"})
genderDemo_df["Percentage of Players"] = (genderDemo_df["Total Count"] / totalPlayers).map("{:,.2%}".format)
genderDemo_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [7]:
# Purchasing Analysis (Gender)

# Create groupby object by Gender
genderGrouped = purchaseData_df.groupby("Gender")

# Number of items purchased by gender group
gendPurchCount = genderGrouped.count()["Item ID"]

# Average item price by gender group
avgGendPurchPrice = genderGrouped.mean()["Price"]

# Sum purchases by gender group
gendTotalPurch = genderGrouped.sum()["Price"]

# Average total purchase by person by gender group
gendAvgPurchPerson = gendTotalPurch / genderDemo_df["Total Count"]

# Display in dataframe
purchGender_df = pd.DataFrame({
    "Purchase Count": gendPurchCount, 
    "Average Purchase Price": avgGendPurchPrice, 
    "Total Purchase Value": gendTotalPurch,
    "Avg Total Purchase per Person": gendAvgPurchPerson
})

# Format dataframe with $
final_purchGender_df = purchGender_df.copy()
final_purchGender_df["Average Purchase Price"] = purchGender_df["Average Purchase Price"].map("${:,.2f}".format)
final_purchGender_df["Total Purchase Value"] = purchGender_df["Total Purchase Value"].map("${:,.2f}".format)
final_purchGender_df["Avg Total Purchase per Person"] = purchGender_df["Avg Total Purchase per Person"].map("${:,.2f}".format)

# Display formatted summary dataframe
final_purchGender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [8]:
# Age Demographics Analysis

# Find youngest and oldest ages to determine bin sizes
yngest = purchaseData_df["Age"].min()
oldest = purchaseData_df["Age"].max()
print(yngest)
print(oldest)

7
45


In [9]:
# Create bins and labels
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.0, 99]
binLabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Cut purchase dataframe with bins and add binned ages as new column in dataframe
purchaseData_df["Age Group"] = pd.cut(purchaseData_df["Age"], bins, labels=binLabels)
# purchaseData_df.head()

# Create groupby object by age bins
ageGroups = purchaseData_df.groupby("Age Group")

# Count by each age group
countOfAgeGroups = ageGroups["SN"].nunique()

# Calculate percent of total players by age group
percentByAgeGroups = countOfAgeGroups / totalPlayers

# Display in dataframe
ageDemo_df = pd.DataFrame({
    "Total Count": countOfAgeGroups, 
    "Percentage of Players": percentByAgeGroups
})

# Format dataframe with %
final_ageDemo_df = ageDemo_df.copy()
final_ageDemo_df["Percentage of Players"] = ageDemo_df["Percentage of Players"].map("{:,.2%}".format)

# Display formatted summary dataframe
final_ageDemo_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [10]:
# Purchasing Analysis by Age

# Use ageGroups groupby object with data already grouped by age bin

# Purchase count by age group
purchCountByAge = ageGroups["Item ID"].count()

# Average purchase price by age group
avgPurchPriceByAge = ageGroups["Price"].mean()

# Sum purchases by age group
totalPurchByAge = ageGroups["Price"].sum()

# Average total purchase by person by age group
avgPurchPersonByAge = totalPurchByAge / countOfAgeGroups

# Display in dataframe
purchByAge_df = pd.DataFrame({
    "Purchase Count": purchCountByAge, 
    "Average Purchase Price": avgPurchPriceByAge,
    "Total Purchase Value": totalPurchByAge, 
    "Avg Total Purchase per Person": avgPurchPersonByAge
})

# Format dataframe with $
final_purchByAge_df = purchByAge_df.copy()
final_purchByAge_df["Average Purchase Price"] = purchByAge_df["Average Purchase Price"].map("${:,.2f}".format)
final_purchByAge_df["Total Purchase Value"] = purchByAge_df["Total Purchase Value"].map("${:,.2f}".format)
final_purchByAge_df["Avg Total Purchase per Person"] = purchByAge_df["Avg Total Purchase per Person"].map("${:,.2f}".format)

# Display formatted summary dataframe
final_purchByAge_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [11]:
# Top Spenders Analysis

# Create groupby object by SN
snGrouped = purchaseData_df.groupby("SN")

# Purchase count by SN grouped
countpurchBySN = snGrouped["Item ID"].nunique()

# Average purchase price by SN grouped
avgPurchPriceBySN = snGrouped["Price"].mean()

# Sum purchases by SN grouped
totalPurchBySN = snGrouped["Price"].sum()

# Make dataframe
purchBySN_df = pd.DataFrame({
    "Purchase Count": countpurchBySN, 
    "Average Purchase Price": avgPurchPriceBySN,
    "Total Purchase Value": totalPurchBySN
})

# Sort Total Purchase Values in descending order
topPurchBySN_df = purchBySN_df.sort_values("Total Purchase Value", ascending=False)

# Foramt dataframe with $
final_topPurchBySN_df = topPurchBySN_df.copy()
final_topPurchBySN_df["Average Purchase Price"] = topPurchBySN_df["Average Purchase Price"].map("${:,.2f}".format)
final_topPurchBySN_df["Total Purchase Value"] = topPurchBySN_df["Total Purchase Value"].map("${:,.2f}".format)

# Display top 5 spenders
final_topPurchBySN_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [20]:
# Add gender and age to top spender analysis 
topPurchBySNWithGender_df = pd.merge(final_topPurchBySN_df, noDuplicateGender_df, right_on="SN", left_index=True, how="left")

# Reorder columns
reorderTopPurchGenders_df = topPurchBySNWithGender_df[['SN', 'Gender', 'Age', 'Purchase Count', 
                                                   'Average Purchase Price', 'Total Purchase Value']]

# Set SN as index
setSNIndex_df = reorderTopPurchGenders_df.set_index('SN')
setSNIndex_df.head(10)

,Gender,Age,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,,,
Lisosia93,Male,25,5,$3.79,$18.96
Idastidru52,Male,24,4,$3.86,$15.45
Chamjask73,Female,22,3,$4.61,$13.83
Iral74,Male,21,4,$3.40,$13.62
Iskadarya95,Male,20,3,$4.37,$13.10
Ilarin91,Male,22,3,$4.23,$12.70
Ialallo29,Female,15,3,$3.95,$11.84
Tyidaim51,Female,16,3,$3.94,$11.83
Lassilsala30,Male,21,3,$3.84,$11.51


In [ ]:
# Most Popular Items Analysis

# Isolate columns related to items in new dataframe
item_df = purchaseData_df[["Item ID", "Item Name", "Price"]]

# Create groupby object by Item ID and Item Name
itemsGrouped = item_df.groupby(["Item ID", "Item Name"])

# Purchase count by items grouped
countpurchByItem = itemsGrouped["Price"].count()

# Sum purchases by items grouped
totalPurchByItem = itemsGrouped["Price"].sum()

# Calculate item price since groupby object has items grouped
itemPricebyItem = totalPurchByItem / countpurchByItem

# Display in dataframe
purchByItem_df = pd.DataFrame({
    "Purchase Count": countpurchByItem, 
    "Item Price": itemPricebyItem,
    "Total Purchase Value": totalPurchByItem
})

# Sort Purchase Count column in descending order 
topItemsByPurchCount_df = purchByItem_df.sort_values("Purchase Count", ascending=False)
# topItemsByItems_df.head(15)

# Format dataframe with $
final_topItemsByPurchCount_df = topItemsByPurchCount_df.copy()
final_topItemsByPurchCount_df["Item Price"] = topItemsByPurchCount_df["Item Price"].map("${:,.2f}".format)
final_topItemsByPurchCount_df["Total Purchase Value"] = topItemsByPurchCount_df["Total Purchase Value"].map("${:,.2f}".format)

# Display 5 most popular item by purchase count
final_topItemsByPurchCount_df.head(10)

In [ ]:
# Most Profitable Items Analysis

# Sort Total Purchase Value column in descending order 
topItemsByTotalPurchVal_df = purchByItem_df.sort_values("Total Purchase Value", ascending=False)
# topItemsByTotalPurchVal_df.head(15)

# Format dataframe with $
final_topItemsByTotalPurchVal_df = topItemsByTotalPurchVal_df.copy()
final_topItemsByTotalPurchVal_df["Item Price"] = topItemsByTotalPurchVal_df["Item Price"].map("${:,.2f}".format)
final_topItemsByTotalPurchVal_df["Total Purchase Value"] = topItemsByTotalPurchVal_df["Total Purchase Value"].map("${:,.2f}".format)

# Display 5 most popular item by purchase count
final_topItemsByTotalPurchVal_df.head(10)